In [1]:
import cv2  
from matplotlib import pyplot as plt

In [2]:
import os
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import torchvision.transforms as transforms

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
import numpy as np
import random

In [3]:
from frame import CELEBADriver, WFDriver, MTCDataSet

In [4]:
# 定义转换操作
transform = transforms.Compose([
    transforms.Resize(12),
    transforms.CenterCrop(12),
    transforms.ToTensor(),  # 将PIL图像或NumPy ndarray转换为FloatTensor。
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # 标准化，使用ImageNet的均值和标准差
                         std=[0.229, 0.224, 0.225])
])

In [5]:

bbox_path = r"C:\Users\lucyc\Desktop\celebA\list_bbox_celeba.csv"
ldmk_path = r"C:\Users\lucyc\Desktop\celebA\list_landmarks_align_celeba.csv"
basic_path = r"C:\Users\lucyc\Desktop\celebA\img_align_celeba\img_align_celeba"

cead = CELEBADriver(bbox_path, ldmk_path, basic_path)

mat_path = r"C:\Users\lucyc\Desktop\faces\WIDER_train\WIDER_train\images"
clas_root_path = r"C:\Users\lucyc\Desktop\faces\wider_face_split\wider_face_split\wider_face_train.mat"

wfd = WFDriver(clas_root_path, mat_path)


In [6]:
cead.random_init()
wfd.random_init()

!!! SERIOUS RANDOM INIT DATASET ALARM !!! type ‘y’ to continue...  y


RANDOM SET


!!! SERIOUS RANDOM INIT DATASET ALARM !!! type ‘y’ to continue...  y


RANDOM SET


In [7]:
len(wfd.dataset_index)

12880

In [8]:
# 创建数据集
test_dataset = MTCDataSet(wfd, cead, [10304,12870], transform)
val_dataset = MTCDataSet(wfd, cead, [7728,10304], transform)
train_dataset = MTCDataSet(wfd, cead, [0,7728], transform)

In [9]:
# 创建数据加载器
train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True, num_workers=3, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=True, num_workers=3, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=True, num_workers=3, pin_memory=True)

In [10]:
a = next(iter(val_loader))

ValueError: Caught ValueError in DataLoader worker process 0.
Original Traceback (most recent call last):
  File "C:\Users\lucyc\anaconda3\envs\torch\Lib\site-packages\torch\utils\data\_utils\worker.py", line 308, in _worker_loop
    data = fetcher.fetch(index)
           ^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lucyc\anaconda3\envs\torch\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in fetch
    data = [self.dataset[idx] for idx in possibly_batched_index]
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\lucyc\anaconda3\envs\torch\Lib\site-packages\torch\utils\data\_utils\fetch.py", line 51, in <listcomp>
    data = [self.dataset[idx] for idx in possibly_batched_index]
            ~~~~~~~~~~~~^^^^^
  File "C:\Users\lucyc\Desktop\AI_GOGOGO\Pytorch\face_loc\frame.py", line 167, in __getitem__
    for img, bbox in self.wfd_random_cut(wfd_img, wfd_bbox, type_of_sample):
                     ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^

In [ ]:
b = next(iter(val_loader))

In [ ]:
c = next(iter(val_loader))

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "CPU")
print(device)
torch.cuda.empty_cache()

In [16]:

mat = scipy.io.loadmat(r"C:\Users\lucyc\Desktop\faces\wider_face_split\wider_face_split\wider_face_train.mat")


In [18]:
mat.keys()

dict_keys(['__header__', '__version__', '__globals__', 'blur_label_list', 'event_list', 'expression_label_list', 'face_bbx_list', 'file_list', 'illumination_label_list', 'invalid_label_list', 'occlusion_label_list', 'pose_label_list'])

In [57]:
mat["face_bbx_list"][0].shape

(1,)

In [164]:
wfd = WFDriver(r"C:\Users\lucyc\Desktop\faces\wider_face_split\wider_face_split\wider_face_train.mat",
               r"C:\Users\lucyc\Desktop\faces\WIDER_train\WIDER_train\images")

In [165]:
wfd.get_file_path(61,2)

'C:\\Users\\lucyc\\Desktop\\faces\\WIDER_train\\WIDER_train\\images\\61--Street_Battle\\61_Street_Battle_streetfight_61_382.jpg'

array([[223, 289,  32,  44],
       [357, 258,  20,  27],
       [411, 313,  18,  26],
       [346, 224,  11,  15],
       [298, 224,  15,  20],
       [193, 244,  15,  14],
       [223, 232,  12,  14],
       [135, 239,  26,  35],
       [ 86, 222,   7,   8],
       [100, 231,   6,   8],
       [176, 232,   8,  10],
       [389, 238,  10,  14],
       [474, 212,  12,  13],
       [ 16, 225,   9,  11]])